# Final cube analysis

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

%matplotlib notebook
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets


import pickle

name_data_file = 'newExtrapolated_allCorrection.pickle'

with open(name_data_file, "rb") as input_file:
    data = pickle.load(input_file)

data.keys()

dict_keys(['phis', 'gams', 'thes', 'geoCUBE', 'potCube', 'smoCube', 'kinCube', 'dipCUBE'])

## Coordinates

In [7]:
from quantumpropagator import fromLabelsToFloats, labTranformA

phis_ext = labTranformA(data['phis'])
gams_ext = labTranformA(data['gams'])
thes_ext = labTranformA(data['thes'])

phiV_ext, gamV_ext, theV_ext = fromLabelsToFloats(data)

# take step
dphi = phis_ext[0] - phis_ext[1]
dgam = gams_ext[0] - gams_ext[1]
dthe = thes_ext[0] - thes_ext[1]

# take range
range_phi = phis_ext[-1] - phis_ext[0]
range_gam = gams_ext[-1] - gams_ext[0]
range_the = thes_ext[-1] - thes_ext[0]


header = '         Labels extr.        internal extr.     dq      range\n'
string = 'Phi -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}\nGam -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}\nThe -> {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f} {:8.4f}'
out = (header + string).format(phiV_ext[-1],phiV_ext[0],phis_ext[-1],phis_ext[0],dphi,range_phi,
                               gamV_ext[-1],gamV_ext[0],gams_ext[-1],gams_ext[0],dgam,range_gam,
                               theV_ext[-1],theV_ext[0],thes_ext[-1],thes_ext[0],dthe,range_the)
print(out)

         Labels extr.        internal extr.     dq      range
Phi ->   0.1250  -0.1450  12.5000 -14.5000  -0.5000  27.0000
Gam ->   0.4294   0.1344  24.6050   7.7030  -0.3070  16.9020
The ->   0.5806   1.2008  66.5340 137.6000   0.4470 -71.0660


## NACS visualization

In [3]:
nacs = data['smoCube']

In [4]:
# take out zeros
NACS = nacs[15:-15,15:-15,30:-30]
phis = phis_ext[15:-15]
gams = gams_ext[15:-15]
thes = thes_ext[30:-30]
phiV = phiV_ext[15:-15]
gamV = gamV_ext[15:-15]
theV = theV_ext[30:-30]
# select the two states
NACS.shape, nacs.shape

((25, 26, 100, 8, 8, 3), (55, 56, 160, 8, 8, 3))

In [5]:
def do3dplot(xs,ys,zss):
    'with mesh function'
    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(111, projection='3d')
    X,Y = np.meshgrid(ys,xs)
    #ax.set_zlim(-1, 1)
    #ax.scatter(X, Y, zss)
    ax.plot_wireframe(X, Y, zss)
    fig.tight_layout()

def visualize_this_thing(thing,state1,state2,cart,kind,dim):
    print(thing.shape)

    along = ['Phi','Gam','The']
    print('NAC between state ({},{}) along {}\nDoing cut in {} with value ({:8.4f},{:8.4f})'.format(state1,
                                                                state2,
                                                                along[cart],
                                                                kind,
                                                                dimV[kind][dim],
                                                                dims[kind][dim]))
    if kind == 'Phi':
        pot = thing[dim,:,:,state1,state2,cart]
        print('\nLooking at SMO with indexes [{},:,:,{},{},{}]'.format(dim, state1,state2,cart))
        do3dplot(gams,thes,pot)
    elif kind == 'Gam':
        print('\nLooking at SMO with indexes [:,{},:,{},{},{}]'.format(dim, state1,state2,cart))
        pot = thing[:,dim,:,state1,state2,cart]
        do3dplot(phis,thes,pot)
    elif kind == 'The':
        print('\nLooking at SMO with indexes [:,:,{},{},{},{}]'.format(dim, state1,state2,cart))
        pot = thing[:,:,dim,state1,state2,cart]
        do3dplot(phis,gams,pot)


dimV = { 'Phi': phiV, 'Gam': gamV, 'The': theV } # real values
dims = { 'Phi': phis, 'Gam': gams, 'The': thes } # for labels

kinds = ['Phi','Gam','The']

def fun_pot2D(kind,state1, state2, cart,dim):
    visualize_this_thing(NACS, state1, state2, cart, kind, dim)
    
def nested(kinds):
    dimensionV = dimV[kinds]
    interact(fun_pot2D, kind=fixed(kinds), state1 = widgets.IntSlider(min=0,max=7,step=1,value=0), state2 = widgets.IntSlider(min=0,max=7,step=1,value=0), cart = widgets.IntSlider(min=0,max=2,step=1,value=0), dim = widgets.IntSlider(min=0,max=(len(dimensionV)-1),step=1,value=0))

interact(nested, kinds = ['Phi','Gam','The']);

A Jupyter Widget

In [6]:
data['geoCUBE'].shape

(55, 56, 160, 15, 3)

# DIPOLES visualization

In [8]:
dipo = data['dipCUBE']
DIPO = dipo[15:-15,15:-15,30:-30]
dipo.shape, DIPO.shape

((55, 56, 160, 3, 8, 8), (25, 26, 100, 3, 8, 8))

In [9]:
def do3dplot(xs,ys,zss):
    'with mesh function'
    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(111, projection='3d')
    X,Y = np.meshgrid(ys,xs)
    #ax.set_zlim(-1, 1)
    #ax.scatter(X, Y, zss)
    ax.plot_wireframe(X, Y, zss)
    fig.tight_layout()

def visualize_this_thing(thing,state1,state2,cart,kind,dim):
    print(thing.shape)

    along = ['X','Y','Z']
    print('DIPOLE between state ({},{}) along {}\nDoing cut in {} with value ({:8.4f},{:8.4f})'.format(state1,
                                                                state2,
                                                                along[cart],
                                                                kind,
                                                                dimV[kind][dim],
                                                                dims[kind][dim]))
    if kind == 'Phi':
        pot = thing[dim,:,:,cart,state1,state2]
        print('\nLooking at DIPOLE with indexes [{},:,:,{},{},{}]'.format(dim,cart,state1,state2))
        do3dplot(gams,thes,pot)
    elif kind == 'Gam':
        print('\nLooking at DIPOLE with indexes [:,{},:,{},{},{}]'.format(dim,cart,state1,state2))
        pot = thing[:,dim,:,cart,state1,state2]
        do3dplot(phis,thes,pot)
    elif kind == 'The':
        print('\nLooking at DIPOLE with indexes [:,:,{},{},{},{}]'.format(dim,cart,state1,state2))
        pot = thing[:,:,dim,cart,state1,state2]
        do3dplot(phis,gams,pot)


dimV = { 'Phi': phiV, 'Gam': gamV, 'The': theV } # real values
dims = { 'Phi': phis, 'Gam': gams, 'The': thes } # for labels

kinds = ['Phi','Gam','The']

def fun_pot2D(kind,state1, state2, cart,dim):
    visualize_this_thing(DIPO, state1, state2, cart, kind, dim)
    
def nested(kinds):
    dimensionV = dimV[kinds]
    interact(fun_pot2D, kind=fixed(kinds), state1 = widgets.IntSlider(min=0,max=7,step=1,value=0), state2 = widgets.IntSlider(min=0,max=7,step=1,value=0), cart = widgets.IntSlider(min=0,max=2,step=1,value=0), dim = widgets.IntSlider(min=0,max=(len(dimensionV)-1),step=1,value=0))

interact(nested, kinds = ['Phi','Gam','The']);

A Jupyter Widget

In [10]:
pot = data['potCube'] - data['potCube'].min()
A = pot
# find the minimum index having the shape
np.unravel_index(A.argmin(), A.shape)

(29, 28, 55, 0)

In [10]:
NACS.shape

(25, 26, 100, 8, 8, 3)

In [11]:
B = NACS[1,:,:,0,1,0]

In [12]:
index = np.unravel_index(B.argmax(), B.shape)

In [13]:
index, B[index], NACS[1,7,81,0,1,0]

((7, 81), 4.605073601526367e-05, 4.605073601526367e-05)

In [14]:
nacs[16,22,111,0,1,0]

4.605073601526367e-05

## temporary cells for last correction sign

In [15]:
# npy = '/home/alessio/Desktop/NAC_CORRECTION_NOVEMBER2018/dataprova.npy'
# dictio = np.load(npy)[()]
# dictio.keys()

In [16]:
# NACS2 = dictio['nacCUBE']
# NACS2.shape

In [17]:
# def do3dplot(xs,ys,zss):
#     'with mesh function'
#     fig = plt.figure(figsize=(9,9))
#     ax = fig.add_subplot(111, projection='3d')
#     X,Y = np.meshgrid(ys,xs)
#     #ax.set_zlim(-1, 1)
#     #ax.scatter(X, Y, zss)
#     ax.plot_wireframe(X, Y, zss)
#     fig.tight_layout()

# def visualize_this_thing(thing,state1,state2,cart,kind,dim):
#     print(thing.shape)
#     print('\nWARNING, this is not fully correct!!! Not SMO and not really what you think\n')

#     along = ['Phi','Gam','The']
#     print('NAC between state ({},{}) along {}\nDoing cut in {} with value ({:8.4f},{:8.4f})'.format(state1,
#                                                                 state2,
#                                                                 along[cart],
#                                                                 kind,
#                                                                 dimV[kind][dim],
#                                                                 dims[kind][dim]))
#     if kind == 'Phi':
#         pot = thing[dim,:,:,state1,state2,0,cart]
#         print('\nLooking at SMO with indexes [{},:,:,{},{},{}]'.format(dim, state1,state2,cart))
#         do3dplot(gams,thes,pot)
#     elif kind == 'Gam':
#         print('\nLooking at SMO with indexes [:,{},:,{},{},{}]'.format(dim, state1,state2,cart))
#         pot = thing[:,dim,:,state1,state2,0,cart]
#         do3dplot(phis,thes,pot)
#     elif kind == 'The':
#         print('\nLooking at SMO with indexes [:,:,{},{},{},{}]'.format(dim, state1,state2,cart))
#         pot = thing[:,:,dim,state1,state2,0,cart]
#         do3dplot(phis,gams,pot)


# dimV = { 'Phi': phiV, 'Gam': gamV, 'The': theV } # real values
# dims = { 'Phi': phis, 'Gam': gams, 'The': thes } # for labels

# kinds = ['Phi','Gam','The']

# def fun_pot2D(kind,state1, state2, cart,dim):
#     visualize_this_thing(NACS2, state1, state2, cart, kind, dim)
    
# def nested(kinds):
#     dimensionV = dimV[kinds]
#     interact(fun_pot2D, kind=fixed(kinds), state1 = widgets.IntSlider(min=0,max=7,step=1,value=0), state2 = widgets.IntSlider(min=0,max=7,step=1,value=0), cart = widgets.IntSlider(min=0,max=2,step=1,value=0), dim = widgets.IntSlider(min=0,max=(len(dimensionV)-1),step=1,value=0))

# interact(nested, kinds = ['Phi','Gam','The']);

# sign flipper on extrapolated SMO cube
you used the cells below to correct NAC on the main plane... it was still flipping

In [18]:
data.keys()

dict_keys(['kinCube', 'thes', 'potCube', 'phis', 'geoCUBE', 'smoCube', 'gams', 'dipCUBE'])

In [19]:
data_new = data

In [20]:
nacs_new = data['smoCube']
NACS_new = nacs_new[15:-15,15:-15,30:-30]
NACS_new.shape,nacs_new.shape

((25, 26, 100, 8, 8, 3), (55, 56, 160, 8, 8, 3))

In [21]:
phi_ext_000_000 = 29
phi_prev = 28
phi_next = 30

new_nacs = np.copy(nacs)

for g in range(56):
    for t in range(160):
        not_correct  = nacs[phi_ext_000_000,g,t]
        correct_prev = nacs[phi_prev       ,g,t]
        correct_next = nacs[phi_next       ,g,t]
        #if np.linalg.norm(not_correct) > 0.001:
        #        print('{} {}\nThis {} \nMiddle {}\n After {}'.format(g,t,correct_prev[:,:,1], not_correct[:,:,1],correct_next[:,:,1]))
        for state1 in range(8):
            for state2 in range(8):
                for cart in range(3):
                    value_prev = correct_prev[state1,state2,cart]
                    value_this = not_correct [state1,state2,cart]
                    value_next = correct_next[state1,state2,cart]
                    average = (value_prev + value_next)/2
                    if np.sign(average) == np.sign(value_this):
                        new_value = value_this
                    else:
                        new_value = -value_this 
                    new_nacs[phi_ext_000_000,g,t,state1,state2,cart] = new_value


#                    if np.linalg.norm(not_correct[state1,state2,:]) > 0.0001:
#                    print('{} {} {}, ({},{})\nThis   {}\nMiddle {}\n After {}'.format(g,t,not_correct.shape,state1,state2,correct_prev[state1,state2,cart], not_correct[state1,state2,],correct_next[state1,state2,:]))


In [22]:
def do3dplot(xs,ys,zss):
    'with mesh function'
    fig = plt.figure(figsize=(9,9))
    ax = fig.add_subplot(111, projection='3d')
    X,Y = np.meshgrid(ys,xs)
    #ax.set_zlim(-1, 1)
    #ax.scatter(X, Y, zss)
    ax.plot_wireframe(X, Y, zss)
    fig.tight_layout()

def visualize_this_thing(thing,state1,state2,cart,kind,dim):
    print(thing.shape)

    along = ['Phi','Gam','The']
    print('NAC between state ({},{}) along {}\nDoing cut in {} with value ({:8.4f},{:8.4f})'.format(state1,
                                                                state2,
                                                                along[cart],
                                                                kind,
                                                                dimV[kind][dim],
                                                                dims[kind][dim]))
    if kind == 'Phi':
        pot = thing[dim,:,:,state1,state2,cart]
        print('\nLooking at SMO with indexes [{},:,:,{},{},{}]'.format(dim, state1,state2,cart))
        do3dplot(gams_ext,thes_ext,pot)
    elif kind == 'Gam':
        print('\nLooking at SMO with indexes [:,{},:,{},{},{}]'.format(dim, state1,state2,cart))
        pot = thing[:,dim,:,state1,state2,cart]
        do3dplot(phis_ext,thes_ext,pot)
    elif kind == 'The':
        print('\nLooking at SMO with indexes [:,:,{},{},{},{}]'.format(dim, state1,state2,cart))
        pot = thing[:,:,dim,state1,state2,cart]
        do3dplot(phis_ext,gams_ext,pot)


dimV = { 'Phi': phiV_ext, 'Gam': gamV_ext, 'The': theV_ext } # real values
dims = { 'Phi': phis_ext, 'Gam': gams_ext, 'The': thes_ext } # for labels

kinds = ['Phi','Gam','The']

def fun_pot2D(kind,state1, state2, cart,dim):
    visualize_this_thing(new_nacs, state1, state2, cart, kind, dim)
    
def nested(kinds):
    dimensionV = dimV[kinds]
    interact(fun_pot2D, kind=fixed(kinds), state1 = widgets.IntSlider(min=0,max=7,step=1,value=0), state2 = widgets.IntSlider(min=0,max=7,step=1,value=0), cart = widgets.IntSlider(min=0,max=2,step=1,value=0), dim = widgets.IntSlider(min=0,max=(len(dimensionV)-1),step=1,value=0))

interact(nested, kinds = ['Phi','Gam','The']);

A Jupyter Widget

In [23]:
name_data_file_new = 'newExtrapolated_allCorrectionSECOND.pickle'
data_new.keys()

dict_keys(['kinCube', 'thes', 'potCube', 'phis', 'geoCUBE', 'smoCube', 'gams', 'dipCUBE'])

In [24]:

data_new['smoCube'] = new_nacs

pickle.dump( data_new, open( name_data_file_new, "wb" ) )